In [1]:
from amuse.units import units
from amuse.lab import Huayno, nbody_system, new_galactics_model
from amuse.lab import Gadget2
from amuse.community.ph4.interface import ph4
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from amuse.couple import bridge
from amuse.community.hermite.interface import Hermite
from amuse.community.bhtree.interface import BHTree
from astropy.io import fits

Initialize the galaxies

In [2]:
%%time


n_halo  = 20000
n_bulge = 10000
n_disk  = 10000
M_galaxy = 1e12 | units.MSun
R_galaxy = 80  | units.kpc
converter = nbody_system.nbody_to_si(M_galaxy, R_galaxy)

galaxy1 = new_galactics_model(n_halo,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
galaxy1.move_to_center()
# Order of particles: Disk => Bulge => Halo

galaxy2 = new_galactics_model(n_halo,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
galaxy2.move_to_center()

n_halo_test  = 20000
n_disk_test  = 10000
n_bulge_test = 10000

test_particles_1 = new_galactics_model(n_halo_test,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test,
                                disk_number_of_particles=n_disk_test)

test_particles_1.move_to_center()
test_particles_1.mass = 0 |units.kg

test_particles_2 = new_galactics_model(n_halo_test,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test,
                                disk_number_of_particles=n_disk_test)

test_particles_2.move_to_center()
test_particles_2.mass = 0 |units.kg

test_particles_1_stars = test_particles_1[:int(n_bulge_test+n_disk_test)]
test_particles_2_stars = test_particles_2[:int(n_bulge_test+n_disk_test)]

CPU times: user 55.9 s, sys: 9.02 s, total: 1min 4s
Wall time: 1min 8s


We rotate the test particles by 180 degrees in the x & y direction to make sure that the particles are not on top of the stars of the galaxies

In [3]:
test_particles_1_stars.rotate(np.pi,np.pi,0)
test_particles_2_stars.rotate(np.pi,np.pi,0)

move and rotate the galaxies to their appropiate initial conditions

In [4]:
galaxy1.rotate(0., np.pi/2, 0.)
galaxy2.x  += 400 | units.kpc
galaxy2.vx += -500 |units.kms

test_particles_1_stars.rotate(0., np.pi/2, 0.)
test_particles_2_stars.x  += 400 | units.kpc
test_particles_2.vx += -500 |units.kms

Create the the gravity solvers and add the particles

In [5]:
converter = nbody_system.nbody_to_si(1e12|units.MSun, 100|units.kpc)
dynamics = BHTree(converter) 
dynamics.parameters.epsilon_squared = (100|units.parsec)**2
dynamics.parameters.timestep = 1 |units.Myr
set1 = dynamics.particles.add_particles(galaxy1)
set2 = dynamics.particles.add_particles(galaxy2)

star_dynamics = BHTree(converter)
star_set_1 = star_dynamics.particles.add_particles(test_particles_1_stars)
star_set_2 = star_dynamics.particles.add_particles(test_particles_2_stars)
star_dynamics.parameters.timestep = 1|units.Myr

Create the bridge and the channels

In [6]:
gravity = bridge.Bridge(use_threading=False)
gravity.add_system(star_dynamics, (dynamics,) )
gravity.add_system(dynamics)
gravity.timestep = 2|units.Myr # needs to be at least twice as large as the individual timesteps
channel_stars = star_dynamics.particles.new_channel_to(star_dynamics.particles)
channel_galaxies = dynamics.particles.new_channel_to(dynamics.particles)

The loop that evolves the gravity code

In [ ]:
times = np.arange(0., 2001, 2) | units.Myr
threshold = 10. |units.Myr 
for time in tqdm(range(len(times))):
    dynamics.evolve_model(times[time])
    if times[time] %threshold == 0|units.Myr:
        channel_galaxies.copy()
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.y[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.y[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[:int(n_disk)].value_in(units.kpc), set2.y[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set2.y[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("test_plots_xy/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[:int(n_disk)].value_in(units.kpc), set2.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set2.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("test_plots_xz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        channel_stars.copy()
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.y[:int(n_bulge_test)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_1.y[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_2.x[:int(n_disk_test)].value_in(units.kpc), star_set_2.y[:int(n_bulge_test)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_2.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_2.y[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("test_plots_stars_xy/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.z[:int(n_bulge_test)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_1.z[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_2.x[:int(n_disk_test)].value_in(units.kpc), star_set_2.z[:int(n_bulge_test)].value_in(units.kpc), s=0.3)
        plt.scatter(star_set_2.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_2.z[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("test_plots_stars_xz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        file = open("time.txt","w")
        file.write(str(times[time]))
        file.write("\n")
        #put the last data in fits files
        col1 = fits.Column(name='star_1_x', format='E', array=np.array(star_set_1.x.value_in(units.m)))
        col2 = fits.Column(name='star_1_y', format='E', array=np.array(star_set_1.y.value_in(units.m)))
        col3 = fits.Column(name='star_1_z', format='E', array=np.array(star_set_1.z.value_in(units.m)))
        col4 = fits.Column(name='star_1_vx', format='E', array=np.array(star_set_1.vx.value_in(units.kms)))
        col5 = fits.Column(name='star_1_vy', format='E', array=np.array(star_set_1.vy.value_in(units.kms)))
        col6 = fits.Column(name='star_1_vz', format='E', array=np.array(star_set_1.vz.value_in(units.kms)))

        col11 = fits.Column(name='star_2_x', format='E', array=np.array(star_set_2.x.value_in(units.m)))
        col12 = fits.Column(name='star_2_y', format='E', array=np.array(star_set_2.y.value_in(units.m)))
        col13 = fits.Column(name='star_2_z', format='E', array=np.array(star_set_2.z.value_in(units.m)))
        col14 = fits.Column(name='star_2_vx', format='E', array=np.array(star_set_2.vx.value_in(units.kms)))
        col15 = fits.Column(name='star_2_vy', format='E', array=np.array(star_set_2.vy.value_in(units.kms)))
        col16 = fits.Column(name='star_2_vz', format='E', array=np.array(star_set_2.vz.value_in(units.kms)))
        
        col21 = fits.Column(name='set_1_x', format='E', array=np.array(set1.x.value_in(units.m)))
        col22 = fits.Column(name='set_1_y', format='E', array=np.array(set1.y.value_in(units.m)))
        col23 = fits.Column(name='set_1_z', format='E', array=np.array(set1.z.value_in(units.m)))
        col24 = fits.Column(name='set_1_vx', format='E', array=np.array(set1.vx.value_in(units.kms)))
        col25 = fits.Column(name='set_1_vy', format='E', array=np.array(set1.vy.value_in(units.kms)))
        col26 = fits.Column(name='set_1_vz', format='E', array=np.array(set1.vz.value_in(units.kms)))

        
        col31 = fits.Column(name='set_2_x', format='E', array=np.array(set2.x.value_in(units.m)))
        col32 = fits.Column(name='set_2_y', format='E', array=np.array(set2.y.value_in(units.m)))
        col33 = fits.Column(name='set_2_z', format='E', array=np.array(set2.z.value_in(units.m)))
        col34 = fits.Column(name='set_2_vx', format='E', array=np.array(set2.vx.value_in(units.kms)))
        col35 = fits.Column(name='set_2_vy', format='E', array=np.array(set2.vy.value_in(units.kms)))
        col36 = fits.Column(name='set_2_vz', format='E', array=np.array(set2.vz.value_in(units.kms)))

        cols = fits.ColDefs([col1,col2,col3,col4,col5,col6,col11,col12,col13,col14,col15,col16,col21,col22,col23,col24,col25,col26,col31,col32,col33,col34,col35,col36])
        hdu = fits.BinTableHDU.from_columns(cols)
        hdu.writeto('data_large_set.fits',overwrite =True)
        file.write(str(times[time]))
        file.close()

gravity.stop()

  1%|▏         | 14/1001 [02:53<2:50:24, 10.36s/it]